### Mauer Cory
# CS 614 Assignment 4: Gaming
## Black Jack card counting agent

## Pitch: 
For this project I chose to build a gaming agent to play blackjack. While most other electronic black jack games are implemented using randomly selected cards, I wanted to model an actual deck of cards to see if I could train a model to count cards using the extra state. The goal of this agent is increase odds of winning in blackjack which could be implemented into a competative online blackjack game. 

## Data source:
The data source of this project is a black jack game that I implemented my self. At a high level, there is one modeled deck of cards and cards are handed out to a player and dealer until our deck reaches 17 cards at which point the "deck" is shuffled. the API into the black jack games provides users and models with the following state 
** players sum of card
** dealers sum of cards
** if the player has an ace
** an array showing the number of cards we have seen 1, 2, ... , 10

## Model and data justification:
For building my agent I decided to leverage a deep q model via tensor flow. Since each move of black jack can effectively be treated as a unique hand/game there was no need to model any temporal aspects which led to this relatively simple model. 

In [3]:
    import os
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.optimizers import Adam
    import tensorflow as tf
    state_size = 14
    action_state = 2
    lr = 0.001
    model = Sequential()
    model.add(Dense(32, input_dim=state_size, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(action_state, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=lr))

    model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                480       
                                                                 
 dense_1 (Dense)             (None, 128)               4224      
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 4,962
Trainable params: 4,962
Non-trainable params: 0
_________________________________________________________________


## Commented examples:
TODO

## Testing:
TODO

## Code and run Instructions
TODO

### I agree to sharing this assignment with other students in the course after grading has occurred. 